# Getting started with FortesFit
(Poor person's documentation)

This is a document that introduces the basics of FortesFit along with some code to get you started! It is divided into following sections.
1. [What is FortesFit anyway?](#What-is-FortesFit-anyway?)
    1. [But how does it "fit" a model?](#But-how-does-it-"fit"-a-model?)
    2. [What do you mean FortesFit is "flexible"?](#What-do-you-mean-FortesFit-is-"flexible"?)
    3. [Okay, how do I get started then?](#Okay,-how-do-I-get-started-then?)
2. [Registering filter profiles](#Registering-filter-profiles)
3. [Registering SED models](#Registering-SED-models)
    1. [Using mathematical expression](#Registering-a-model-using-a-mathematical-expression)
    2. [Examining the registered model](#Examining-the-registered-model)
    3. [Using template files](#Registering-a-model-using-a-template-file)
4. [SED fitting](#SED-fitting-with-FortesFit)
    1. [Preparing the data](#Preparing-the-data)
    2. [Performing a fit](#Performing-a-fit)
    3. [Understanding the results](#Understanding-the-results)
5. [Advanced usage](#Advanced-usage)
    1. [Stellar population model](#Stellar-population-model-(Bruzual-and-Charlot,-2003))

Note: This document is in the very initial stages and my first attempt at documenting. So any feedback or suggestions will be much appreciated for improving this in the future. Write to me at: [d.h.liya2@ncl.ac.uk](mailto:d.h.liya2@ncl.ac.uk)

Note 2: I have assumed you have already installed the FortesFit on your machine, set your ```FORTESFITPATH``` variable, [installed pymultinest](https://johannesbuchner.github.io/PyMultiNest/install.html), and ran the initial test script as given on the [GitHub page](https://github.com/vikalibrate/FortesFit). Please [write to me](mailto:d.h.liya2@ncl.ac.uk) if you have trouble in any of those steps!

Note 3: There has been a change in some of the scipy library that has not been incorporated in this public version of FortesFit. To get around this problem, you will need to do the following steps as a temporary solution before we begin the tutorial,

1. Find the installation directory for FortesFit. This is usually in ```~/anaconda3/envs/<environment name>/lib/python3.10/site-packages/fortesfit/``` if you are using conda.
2. Open the file ```FortesFit_Preparation.py``` and replace all three instances of ```rv_frozen``` with ```rv_continuous_frozen```.
3. Save the file and proceed.

# What is FortesFit anyway?

FortesFit is a flexible Spectral Energy Distribution (SED) fitting code that uses Bayesian inference methods to provide robust fits. It uses [Nested Sampling](https://arxiv.org/abs/2101.09675) to explore the complicated parameter sapce which makes it ideal for models with a large number of parameters with complex correlations. 

## But how does it "fit" a model?

On a high-level this is how FortesFit works,
1. Select a random set of model parameters (eg. blackhole mass, accretion rate, etc)
2. Compute the model photometry at these random model parameters. That is, what does the SED look like if this is the correct model?
3. Compare this model photometry to the observed photometry to find out how different they are. (Note this is what "likelihood evaluation" is!)
4. Select a new (semi-)random set of model parameters that are likely to produce an SED closer to the observed SED. 
5. Repeat steps 2 to 4 until the model SED is close enough (under certain definition of close) to the observed SED.
6. This final set of model parameters that produce an SED close to observed SED are what we call "best fit parameters"! 
7. However, from step 3, we "know" how "different" each model SED is from the observed SED. FortesFit actually uses this information to provide the full probability distribution of "best fit parameters" instead of a single value.

## What do you mean FortesFit is "flexible"?

Most SED fitting codes come pre-packaged with a set of observation filters and models. But what if you have a data taken using a shiny new telescope (or a very old telescope) whose filter is not in the SED fitting code? Or what if you have produced your own SED model that you want to fit to the real data? Or what if you want to try and use a different fitting technique? 

FortesFit is your answer for all of these! It gives a complete control over each step of the analysis. Hence, we call it a "flexible SED fitting code". What this means is that there is a bit of work needed, a few decisions to be made before you start. Many of these are a one time decisions but they are totally worth the effort!

## Okay, how do I get started then?

The aforementioned flexibility means that we have to build our own "filter library" and the "model library" before we get to fit a model. Steps to build these libraries are discussed below.

In [ ]:
# import common packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Registering filter profiles
FortesFit maintains an internal "library" of filter profiles. These filter profiles are convolved with a given model (at a given redshift) to produce the observed photometry in that filter. Which is then used as the actual "model" to evaluate likelihood during fitting.

The filter profiles need to be "registered" before they can be used elsewhere in FortesFit. This registration is done using the function ```FortesFit_Filters.register_filter()```. Each filter is assigned a randomly generated unique six digit filter ID during the registration process which can then be used to access the filter.

Tips:
- The filter profiles don't have to be noramlized as they are normalized internally during the registration process.
- It is possible to register multiple instances of the same filter since they are identified only by their filter ID which is unique to each instance.
- [SVO filter profile service](http://svo2.cab.inta-csic.es/theory/fps/) is a good starting point to get most common filter profiles. Although they can sometimes be outdated.
- The fact that the Filter ID is assigned at random means that the filter IDs differ from one computer to another. Therefore, one needs to modify the code while sharing programs. But we won't worry about this for now.

In [ ]:
# import required functions and read docs of register_filter() function

from fortesfit import FortesFit_Filters
FortesFit_Filters.register_filter?

# Questions:
# How many arguments does the register_filter() take?
# What are the required arguments?
# What are the default values of the optional arguments?

In [ ]:
# load and examine one of the filter profiles downloaded from SVO
filter_profile = np.loadtxt('filters/JWST_MIRI.F770W.dat')
print(filter_profile)

plt.plot(filter_profile[:, 0], filter_profile[:, 1])
plt.xlabel('Wavelength (Angstrom)')
plt.ylabel('Throughput')

# Does this look as expected? 

In [ ]:
# register this filter
wavelength = filter_profile[:, 0]/1e4 # !!!! CAUTION !!!!: FortesFit ALWAYS expects wavelength in microns
throughput = filter_profile[:, 1]

# this is a photon counting instrument (usually mentioned in SVO itself)
filter_type = 'photon'

# any string works but I like to use the download location
filter_reference = 'SVO' 

# any other information you'd like to access quickly. I use this as "name of the filter (what does the filter 
# include, eg. Filter and instrument response)"
description = 'JWST MIRI F770W (F + I)' 

# register!
filterid = FortesFit_Filters.register_filter(wavelength=wavelength, throughput=throughput, format=filter_type, 
                                             reference=filter_reference, description=description)
print(f'Successfully registered {description} filter with filter ID {filterid}.')

In [ ]:
# Let's load this filter as the FortesFit "filter object"
fortesfilter = FortesFit_Filters.FortesFit_Filter(filterid)
print(type(fortesfilter))

# we can now access various information from this filter using inbuilt methods
print(f'filter.filterid: {fortesfilter.filterid}')
print(f'filter.description: {fortesfilter.description}')
print(f'filter.format: {fortesfilter.format}')

# we can of course access the actual profile as well! Does this look similar to the original filter?
plt.plot(fortesfilter.wavelength, fortesfilter.throughput)
plt.xlabel('Wavelength (Angstrom)')
plt.ylabel('Throughput')

# Is this same as what we visualised before registering?

In [ ]:
# Register as many filters as you like
filter_profile = np.loadtxt()

wavelength =  # !!!! CAUTION !!!!: FortesFit ALWAYS expects wavelength in microns
throughput = 

filterid = FortesFit_Filters.register_filter(wavelength = wavelength, throughput = throughput, format = , 
                                             reference = , description = )
print(f'Successfully registered {description} filter with filter ID {filterid}.')

**Note: All these filters are listed in a text file ```$FORTESFITPATH/FortesFit_Filters/FortesFit_filters_summary.ascii``` file along with their filter IDs, format, pivot wavelength, and description. Open this file and check!**

In the later part of this tutorial we will look at an actual observed SED of a galaxy taken using 18 different photometric filters. I have downloaded and put all these filters profiles in the folder ```filters/```, and written a small block of code to automatically register all these filters. Go ahead and run the following cell to do this.

In [ ]:
filter_table = pd.read_csv('tutorial_filters.csv')

filter_rootdir = 'filters'

for rownum, row in filter_table.iterrows():
    filterfile = f"{filter_rootdir}/{row['Filename']}"
    original_filter = np.loadtxt(filterfile)
    original_filter[:, 0] = original_filter[:, 0]/1e4
    filter_format = row['Format']
    filter_reference = row['Source']
    filter_description = f"{row['Filtername']} ({row['Includes']})"

    FortesFit_Filters.register_filter(original_filter[:, 0], original_filter[:, 1], format=filter_format,
        reference=filter_reference, description=filter_description)

Check ```$FORTESFITPATH/FortesFit_Filters/FortesFit_filters_summary.ascii``` file to see if these are registered successfully.

# End of part 1!

Congratulations! you have successfully built your own filter library. Take a break and go over anything that is unclear. We will build our model library in the next part.

----

# Registering SED models

- **What is a model?**  
A model is a function that answer the question "What is the observed flux in a given filter for a given set of model parameters and a redshift?"

- **How is a model defined in FortesFit?**  
Any model in FortesFit is defined by a single "scale parameter" and any number (including zero) of "shape parameters". A _scale parameter_ is simply the normalization constant that scales the model flux (eg. IR luminosity), while a _shape parameter_ controls the actual shape of the SED (eg. dust extinction in a galaxy).

Similar to the library of filters, FortesFit maintains a library of models that you need to build during first use. The process of building this library is known as "registration" of models. The real power of FortesFit is in allowing users to register an arbitrary model rather than tying them to a fixed set of models (and parameters!) defined by the developers. Let us register our first model to demonstrate this!

The actual model registration is done using the function ```FortesFit_ModelManagement.register_model()```. However, this function requires a user written script in order to parse the model in a format that FortesFit understands. We call this a "readin script". This provides flexiblity to parse the model from any starting point (eg. numpy array, a bunch of .txt files, a bunch of FITS files, using a mathematical expression, etc). A typical readin script contains two functions. Let us break them down and understand how to write our own!

### 1. The ```readin()``` function
This is a function to calculate SED for a single point in parameter space and at a single redshift. It is a user defined function that can be written in any possible way as long as it respects following rules:
1. Take a dictionary of model parameters as input whose keys are parameter names and the values are floats
2. Take a single redshift as input
3. Return a dictionary with keys ```observed_wavelength``` and ```observed_flux``` whose values are 1D arrays
4. (optionally) Take a set of templates as an input

A general guideline for writing this function is:
1. Obtain the model luminosity at given set of parameters
2. Convert rest frame wavelength to observed wavelengths at given redshift
3. Convert the luminosity into flux using luminosity distance at given redshift ($L/4\pi r^2$)
4. Scale the SED according to the given scale parameter

### 2. The ```readtemplates()``` function
This is an optional function that we will worry about in the next part :)

## Registering a model using a mathematical expression

We will register a simple black body model given by Planck's formula,  


$f_{\lambda} \propto \frac{1}{\lambda^5(e^{hc/kT_{bb}\lambda} - 1)}$  

Where, $f_{\lambda}$ is the flux and $T_{bb}$ is the temperature of the black body. This is a simple but somewhat useful model for FIR emission from the dust. As described before, we will write a function that outputs a single observed frame model SED given a set of parameters and redshift. 

This model will need to be normalised by some scale factor that will scale the SED up and down along the flux axis while fixing the shape of the SED. While we can use arbitrary scaling constant, it is useful to define this constant to be a scientifically useful quantity. For example, the 8-1000 $\mu$m dust luminosity is known to be correlated with the SFR, dust mass and other physically interesting quantities. Therefore, we will normalised our black body SEDs by their 8-1000 $\mu$m luminosity.  

We have all the background to write the ```readin()``` function now. Let's do it!

In [ ]:
from scipy import integrate # we will use this during normalisation
from fortesfit.FortesFit_Settings import FortesFit_Cosmology as cosmo # import cosmology from FortesFit

def readin(parameters, redshift, templates=None):
    """ 
    Black body as a function of wavelength (um) and temperature (K). Returns units of lamdaFlambda.
    
    Inputs:
    ----
    parameters: a dictionary of all parameters (including the scale parameters). They keys are parameter names,
                and the values are floats
    redshift  : single redshift value as a float
    templates : This is an unused variable (in this case) that's necessary for internal workings of FortesFit
                can be safely ignored.
                
    Return:
    ----
    sed       : The observed SED at given set of parameters and redshift as a dictionary of form
                {'observed_wavelength': <list-like>, 'observed_flux': <list-like>}
    """
    # constants in micron units
    h = 6.62607015e-22
    c = 299792458000000.0
    k = 1.380649e-11
    T = parameters['Tbb'] # get temperature of the blackbody from user
    
    # we will normalise our SED to have a certain 8-1000 micron luminosity since it is
    # a useful physical quantity known to correlate with dust mass, SFR etc depending on the application
    user_norm = 10**parameters['logLIR']
    
    # define wavelength range in rest-frame
    # (I think) this samples a good range with enough granularity to catch turn-overs at typical temperatures 
    rest_waves = np.logspace(-2, 3, 1000)
    # also calculate observed frame wavelengths
    observed_waves = rest_waves*(1.0 + redshift)
    
    fl = (1/rest_waves**5)/(np.exp(h*c/(rest_waves*k*T)) - 1) # Flambda in arbitrary units
    lfl = rest_waves*fl # lFl is used for scaling etc
    
    # integrate to find the LIR
    intindex = (rest_waves >= 8.0) & (rest_waves <= 1000.0)
    lir = integrate.trapz(fl[intindex], rest_waves[intindex]) # !!!! NOTE: Integrate over Flambda !!!!
    I_norm = lfl*(user_norm/lir)
    
    # this luminosity will be diluted due to the distance. calculate that factor
    lfactor = 4.0*np.pi*(cosmo.luminosity_distance(redshift).value*3.0856e24)**2.0 #area of spehere in cm^2
    # apply the factor and convert to observed-frame flux
    observed_flux = (I_norm/lfactor)/observed_waves
    
    sed = {'observed_wavelength': observed_waves,'observed_flux': observed_flux}
    
    return sed

# NOTE
# It is always a good idea to do a validation check for evaluating the model at a couple of parameter points 
# and compare with the expected/calculation-by-hand values

# Complete the following piece of code for this purpose
# sed = readin(parameters = {'logLIR': 45, 'Tbb': 100}, redshift=)
# wavelength = sed['observed_wavelength']
# flux = 
# print(f'Flux at observed wavelength {wavelength[]} um is {flux[]} erg/s/cm^2/um')

This function needs to be inside a script to do the actual registration. I have already done this for you in a file called ```blackbody_readin.py```. We can now go ahead and register our first model!

In [ ]:
# load the necessary functions and read the doc string
from fortesfit import FortesFit_ModelManagement
FortesFit_ModelManagement.register_model?

# Questions:
# How many arguments does the register_model() take?
# What are the required arguments?
# What are the default values of the optional arguments?

In [ ]:
readin_filename = 'blackbody_readin' # no need to include .py in the end

# parameters is a dictionary of form {<parameter name>: <parameter value(s)>}
# where the scale parameter must be specified a single value to which all templates will be normalised
# and each shape parameter must be specified by a list-like object whose elements are the values of that parameter
# where model is to be registered (note that these are also called "pivot points" or "model grid". 
# the model is registered only at these points and then interpolated between them during the actual fitting)
parameters = {'logLIR': 45.0, 'Tbb': [20, 30, 40, 50, 60, 70, 80, 100, 120, 140, 150, 200, 500, 1000]}

# name of the scale parameter (must be same as the one in readin() function)
scale_parameter_name = 'logLIR' 

# description is just for the user. use something descriptive that will help you learn more about the model
# i like to use the name, reference if there is, name of the person who wrote the readin, and date of registration
description = 'Blackbody model, Devang (18/07/2024)' 

# all the filter IDs to register this model with. these filters must be registered first (obviously)
# you can add more filters later but the simplest things is to use as many filter here as you can
# include all the extra filters here
filterID_list = [183105, 671789, 766002, 625884, 467664, 992195, 133129, 238244, 987573, 593552, 825551, 109229, 858224, 671851, 514441, 709546, 611585, 142142, 996834] # let's use the same filter that we registered above

# make a list like this from your computer
# filterID_list = [141206, 514919, 101433, 484182, 637257, 606998, 990206, 966425, 431206, 994384]

# wavelength range over which this model is valid (default is 0.01 to 10000.0 micron)
valid_wave_range = [0.1, 10000.0]

# list of redshifts for which to register this model. 
redshift_array = np.array([0.01, 0.02, 0.03, 0.04])
# if not supplied it uses 141 log-uniform points between 0.001 and 10.0
# we will use the default setting here

bbmodelID = FortesFit_ModelManagement.register_model(read_module=readin_filename, parameters=parameters, 
                                                     scale_parameter_name=scale_parameter_name, 
                                                     filterids=filterID_list, wave_range=valid_wave_range,
                                                     description=description)
print(f'Finished the registration of {description} model. The assigned model ID is {bbmodelID}.')

Like filters, all the models are also summariesd in a text file ```$FORTESFITPATH/FortesFit_ModelPhotometry/FortesFit_models_summary.ascii```.

## Examining the registered model

We can plot the model SEDs of the registered model to check if they look as expected. We do this using ```FortesFit_Plots.examine_model_seds()``` function which plots $n$ randomly selected SEDs at the mean redshift for a given model.

In [ ]:
# import plotting function and read docstring
from fortesfit.FortesFit_Plots import examine_model_seds
examine_model_seds?

# Questions:
# How many arguments does the examine_model_seds() take?
# What are the required arguments?
# What are the default values of the optional arguments?

In [ ]:
# let's plot 20 randomly sampled SEDs
examine_model_seds(ModelID = bbmodelID, nsamples = 20)
plt.ylim(1e-16, 1e-12)
# is this what you'd expect from this model?

## Registering a model using a template file

While we can register a model using an analytical expression, often the astrophysical models are more complicated requiring numerical solutions. Therefore, it is much more common for these models to be in form of "template" SEDs that are pre-evaluated at discrete values of model parameters. We will register one such model in this section.

The model we are using comes from [Dale+14](https://ui.adsabs.harvard.edu/abs/2014ApJ...784...83D/abstract) which can be downloaded from [this website](http://physics.uwyo.edu/~ddale/research/seds/seds.html). This is a two parameter IR/radio SED model for hot dust in the galaxy. The two parameters control the temperature of the dust and the AGN fraction in the galaxy. 

For simplicity, we are only going to use pure galaxy SED (AGN fraction = 0). This gives us **one shape parameter** (dust temperature denoted by symbol $\alpha$). The templates are visualized below. 

<img src="dale14_0.00AGN.png" alt="Dale+14 model for 0 AGN fraction" width="700"/>

The scale parameter can be whatever we want. But we are going to use a neat little trick like before to define a physically useful scale parameter. It is known that the total IR luminosity i.e. integrated luminosity between $8-1000 \mu m$ of the galaxy is correlated with the SFR, bolometric luminosity, etc. So, we will use this as our **scale parameter**. Let us see how to do that!

**The ```readtemplates()``` function:**  
We will make use of the ```readtemplates()``` function this time. This is a function used to read templates from the files and parse them for ```readin()``` function. _Technically_, contents of this function can be part of ```readin()```. However, ```readin()``` is called for every single point in parameter space and for every single redshift point. Including these many I/O operations (i.e. reading a file) can make the registration process considerably slower. Therefore, one should always write a ```readtemplates()``` function while reading from an file.

In [ ]:
from scipy import integrate # we will use this during normalisation
from fortesfit.FortesFit_Settings import FortesFit_Cosmology as cosmo # import cosmology from FortesFit

def readtemplates(templateroot=None):
    """
    Read in the templates that will be used by the associated readin routine
    The returned variable can have any user-defined form that is understood by the readin routine below.
    """
    
    # read Dale+14 library from file
    if templateroot is None:
        # !!!!!!! ALWAYS use absolute path !!!!!!!
        # Input the absolute path to the "Dale14" folder distributed with this tutorial
        templateroot = '/Dale14'

    # normalise the SEDs to this 8-1000 micron luminosity
    # choose a value which would be close to typical values encountered during fitting
    l_norm = 1e45

    # values of alpha where the model is evaluated (in this case it's a simple linearly seperated points in alpha)
    AlphaD14       = np.linspace(0.06250, 4, 64)
    template_basic = np.loadtxt(templateroot + '/spectra.0.00AGN.dat') # AGN fraction = 0 template

    wavelength = template_basic[:,0] # base wavelengths are in microns
    seds = np.empty((len(wavelength), len(AlphaD14)))  # empty array which will store the scaled SEDs

    # to normalise, integrate the SED to get the total IR luminosity (8-1000 um)    
    intindex = (wavelength >= 8.0) & (wavelength <= 1000.0)

    # iterate over each alpha value to normalise all templates
    for ialpha, alpha_val in enumerate(AlphaD14):
        template_orig = template_basic[:, ialpha+1] # single template corresponding to the alphad14 value

        # Dale+14 templates are in arbitrary log nu*Fnu
        # to integrate over wavelength, first convert to Flambda
        # units are arbitrary, so simple division by rest wavelength
        integrand = 10**(template_orig[intindex])/wavelength[intindex] 
        lirtot = integrate.trapz(integrand,wavelength[intindex])
            
        # scale the template to log total IR luminosity of 45 erg/s
        seds[:, ialpha] = template_orig + (np.log10(l_norm) - np.log10(lirtot))  
        
    templates = {'wavelength':wavelength, 'seds':seds, 
                 'parameters':{'AlphaD14':AlphaD14, 'GalaxyIRLuminosity': np.log10(l_norm)}}
    
    return templates        
    

# Now let's write the readin() function,

def readin(parameters, redshift, templates=None):
    """ 
    Given a specific parameter set and a redshift, return the Dale+14 pure galaxy template in native units.
        
    Inputs:
    ----
    parameters (dict): A dictionary of model parameters with following keys and single float values
        AlphaD14: The alpha shape parameter for the Dale 2014 library.
        GalaxyIRLuminosity: the 8-1000 um integrated luminosity in log erg/s/cm^2
    redshift (float) : Redshift
    """

    # Templates are needed to process the readin for this class of model. 
    # Catch cases where the template is not supplied.
    if templates == None:
        raise ValueError('Templates must be supplied for this model. Have you written a readtemplates() function?')
    
    # These templates are already normalised to L(8-1000) == 1e45 erg/s, in log nuLnu units.

    wave_basic = templates['wavelength']  #  base wavelengths are in microns
    wave = wave_basic*(1.0 + redshift)  # Observed wavelength 

    param_basic = templates['parameters']['AlphaD14']
    # identify the template index for the given value of alpha
    index = np.argmin(np.abs(param_basic - parameters['AlphaD14']))
    template_orig = 10**(templates['seds'][:, index])
    
    # scale according to the given IR luminosity relative to the one used for normalisation
    scale_factor = 10**(parameters['GalaxyIRLuminosity'] - templates['parameters']['GalaxyIRLuminosity'])
    
    # now convert this to observed flux given a redshift
    lfactor = 4.0*np.pi*(cosmo.luminosity_distance(redshift).value*3.0856e24)**2.0 #area of spehere in cm^2
    observedflux = (template_orig*scale_factor/lfactor)/wave

    sed = {'observed_wavelength':wave,'observed_flux':observedflux}
    
    return sed

# Let's test our functions like before!
lumin = 45
alpha = 2.0
redshift = 1.5
templates = readtemplates()
sed = readin({'AlphaD14': alpha, 'GalaxyIRLuminosity': lumin}, redshift, templates)
wavelength = sed['observed_wavelength']
flux = sed['observed_flux']
print(f'Flux at observed wavelength {wavelength[50]} um is {flux[50]} erg/s/cm^2/um')
# we can also plot the SED
plt.plot(wavelength, wavelength*flux)
plt.xlabel(r'Observed wavelength ($\mu$m)')
plt.ylabel(r'$\lambda F_\lambda$ (erg/s/cm$^2$)')
plt.loglog()

I have put both these functions in a file ```dale14_readin.py``` like before. Use that while registering the model below. **NOTE:** Do not forget to enter the correct path for ```templateroot =``` in this script like you did above.

In [ ]:
readin_filename = 'dale14_readin' # no need to include .py in the end

# We will use all the values of AlphaD14 where the templates are provided
parameters = {'GalaxyIRLuminosity': 45.0, 'AlphaD14': np.linspace(0.06250, 4, 64)}

# name of the scale parameter (must be same as the one in readin() function)
scale_parameter_name = 'GalaxyIRLuminosity'  # what is the scale parameter?

# this is just for the user. use something descriptive that will help you learn more about the model
# i like to use the name, reference if there is, name of the person who wrote the readin, and date of registration
description = 'Pure galaxy dust IR, Dale+14, Devang (18 Jul 2024)' 

# all the filter IDs for which to register this model with
# filterID_list = [filterid]

# wavelength range over which this model is valid (default is 0.01 to 10000.0 micron)
valid_wave_range = [0.1, 1000.0]

# let's skip supplying custom redshifts and use the default range this time

dlmodelID = FortesFit_ModelManagement.register_model(read_module=readin_filename, parameters=parameters, 
                                                     scale_parameter_name=scale_parameter_name, 
                                                     filterids=filterID_list, wave_range=valid_wave_range,
                                                     description=description)

In [ ]:
# let's again plot 50 randomly sampled SEDs to see if the model is registed properly
examine_model_seds(ModelID = dlmodelID, nsamples = 50)
plt.ylim(1e-18, 1e-13)
# is this what you'd expect from this model?

In the next part of the tutorial we will make use of one more SED model that represents the photometric emission from a stellar population in a galaxy.

This is a model with three shape parameters and one scale parameter. The templates for this model are distributed with this tutorail in a folder ```bc03/```. The instructions on how to register this model are in [this section](#Stellar-population-model-(Bruzual-and-Charlot,-2003)) at the end of the tutorial. So go ahead and register that model too! 

Caution: This model can take up to 30min to register.

# End of part 2 !

Congratulations! You have now successfully built your own filter and model libraries. This concludes the "setup" part, and the most difficult part, of FortesFit. You will never require these steps again unless you want to add new filters or models! Give yourself a pat on the back and go drink some coffee maybe :)

----

# SED fitting with FortesFit

In this section, we will learn how to do the SED fitting using FortesFit. This involves [preparing your data](#Preparing-the-data), [performing a fit](#Performing-a-fit), and [understanding the results](#Understanding-the-results). So let's get started!

## Preparing the data

To begin the fitting, we first need to define the things like filters, models, flux, flux errors, and fitting engine. Following section will walk you through each of these steps.

In [ ]:
# import functions necessary for fitting
from astropy.table import Table
import astropy.units as u
from fortesfit import FortesFit_Preparation
from fortesfit import FortesFit_Fitting

In [ ]:
# Source information
source_name = 'Galaxy42'
redshift = 0.912

# Step 1 - Define photometry
# ==========================
phot_table = Table.read('example_sed.txt', format='ascii')
# FortesFit understands different units and can convert between them internally through astropy
# therefore, we'll use astropy.units to assign correct units to our data and let FortesFit worry about conversions
flux = np.array(phot_table['flux(uJy)'])*u.uJy
flux_error = np.array(phot_table['error(uJy)'])*u.uJy

# list of filter IDs for all the filters in our data (this needs to be in correct order!)
filterids = [183105, 671789, 766002, 625884, 467664, 992195, 133129, 238244, 987573, 593552, 825551, 858224, 671851, 514441, 709546, 611585, 142142, 996834]
# --------------------------- Bonus tip ---------------------------
# You can also automate this with lookup table or a simple dictionary! 
# I find it useful to define a "filter_dir" to convert between the name of the filter and the FortesFit filter ID
# filter_dir = {'f115w': 709756, 'f150w': 783907, 'f200w': 125789, 'f277w': 343013, 'f356w': 867636, 
#               'f410m': 546746, 'f444w': 411373, 'f560w': 238422, 'f770w': 620946, 'f1000w': 224853, 
#               'f1280w': 305531, 'f1500w': 523126, 'f1800w': 629076, 
#               'f2100w': 241194} # !! These numbers will be different for you
# filterids = np.array([filter_dir[instrument] for instrument in phot_table['filter']])
# -----------------------------------------------------------------
print(f'[{source_name}, z={redshift}] Provided fluxes: {flux}')
print(f'[{source_name}, z={redshift}] Corresponding flux errors: {flux_error}')
print(f'[{source_name}, z={redshift}] FortesFit filter IDs for each of these: {filterids}')

In [ ]:
# Step 2 (optional) - Assign limits
# ==========================
# NOTE: If no limits are provided then all errors are assumed to be 1-sigma errors by default
# Sometimes you have errors that are 2-sigma or 5-sigma errors
# Sometimes the fluxes are actually limits
# FortesFit deals with all of these cases using a list object called "Limits"
# The elements of this list are:
#    1. Significance (in sigmas) of the errors given in "flux_error" 
#    2. Negative value if the corresponding flux is a limit
# let's start by creating a list of ones, signifying every error is a 1-sigma error
limits = np.ones(flux_error.size)
# let's set the one of the photometric measurements as a limit
limits[14] = -1.0
# and one photometric measurement as 4-sigma limit
limits[1] = 4.0
print(f'[{source_name}, z={redshift}] Using following limits: {limits}')

# Step 3 (optional) - Assign weights
# ==========================
# NOTE: If no weights are provided then all data points are assumed to have same weight by default
# SED fits can sometimes be driven by sharp features in the data
# if these are not real features then they can completely ruin the fit
# similarly, unreliable photometric measurements can also drive the fit in the wrong direction
# all these issues can be dealt with by assigning "lower priority" to problematic photometric point
# i.e. de-weighting them. this is done in FortesFit by using a list of "weights" as described below.
# let's start by creating equal weights (=1) for all points
weights = np.ones(flux_error.size)
# de-weight the FIR photometric measurement (last three points) by a factor of 3
weights[-1] = 1/3.0
weights[-2] = 1/3.0
weights[-3] = 1/3.0
print(f'[{source_name}, z={redshift}] Using following weights: {weights}')

In [ ]:
from scipy.stats import norm, uniform
from astropy.cosmology import Planck15 as cosmo

# Step 4 - Defining the model and assigning priors
# ==========================
# the final model is a sum of one or more components
# eg. total = stars + dust
# the components are specified using a list of model IDs
# let's create a model consisting of power law continum and dust emission
modelids = [67, 76] # <- enter the models IDs you got from registration. 1st is Bruzual&Charlot03, 2nd is Dale+14

# we now need to assign priors to each of the model parameters
# ALL scale parameters MUST have a prior
# if a prior is not assigned to a shape parameter then a uniform prior over the entire range is assumed
# you can fix parameters by providing a single value for prior instead of a distribution
priors_list = []
# -------------- First model (Stellar population, Bruzual & Charlot 03) --------------
priordist = {'modelid': modelids[0]}

# log stellar mass between 8 and 13 covers a reasonable range. we have no idea what the stellar mass of this
# galaxy should be. therefore, we will use a uniform prior between these two values
priordist.update({'logStellarMass': uniform(loc=8.0, scale=5.0)})

# age > 100 Myr, and < age of Universe at that redshift. let us use uniform prior between these two extremes
maxage = cosmo.age(0).value - cosmo.age(redshift).value # maximum look back time: 13.8 - universe age at redshift
minage = 0.1
priordist.update({'age_Gyr': uniform(loc = minage, scale = maxage-minage)})

# we don't have any idea about the lifetime of stellar population in exponential model
# but lets say it is 8.0 and fix the value of this parameter (Note there is not scientific reason for this!!)
priordist.update({'tau_Gyr': 8.0})

# we have no idea for the amount of attenuation of starlight by galaxy dust so we will supply a uniform prior
# between reasonable extreme values
priordist.update({'E_BV_czt': uniform(loc=0.0, scale=2.0)})
priors_list.append(priordist)
# ------------------------------------------------------------------------------------

# --------------------------- Second model (Dust, Dale+14) ---------------------------
priordist = {'modelid': modelids[1]}

# a reasonable range for log galaxy IR luminosity is between 42 and 47 with no preference for any value
# hence, a uniform prior again
priordist.update({'GalaxyIRLuminosity': uniform(loc=42, scale=5)})

# for studies of local galaxy population we find that the median value of Alpha in Dale+14 model is 2
# let us use this knowledge to supply an "informative prior" 
# (i.e. non-uniform prior where certain values are more likely)
# we will do this by using a Gaussian prior with mean = 2 and std = 0.3
priordist.update({'AlphaD14': norm(loc=2.0, scale=0.3)})
priors_list.append(priordist)
# ------------------------------------------------------------------------------------

In [ ]:
# Step 5 - Create FortesFit classes for all these and prepare output files
# ==========================
# the "MinimumError" parameter below controls what is the minimum assumed error on photometry
# if the given error is smaller than this then it is overwritten internally
# this is a good practice when dealing with photometry where the errors are underestimated
data = FortesFit_Preparation.CollectData(ID = source_name, redshift = redshift, 
                                         filterids = filterids, fluxes = flux, fluxerrs = flux_error, 
                                         MinimumError = 10, Weights = weights, Limits = limits)
model = FortesFit_Preparation.CollectModel(modellist = modelids, priordists = priors_list, datacollection = data)
# create output files and folders
# the argument "fitting_method" controls the fitting engine. FortesFit currently supports "emcee" and "multinest"
!mkdir "multinest_output"
fitfile = FortesFit_Preparation.prepare_output_file(datacollection = data, modelcollection = model, 
                                                    fitengine = 'multinest')

## Performing a fit

We are FINALLY ready to do the fit. Somewhat anti-climatically, this is just one line of code!

In [ ]:
!pip install pymultinest

In [ ]:
flatchain = FortesFit_Fitting.FortesFit_FitSingle(datacollection = data, modelcollection = model, fitfile = fitfile,
                                                  outputfiles_basename = 'tutorialfit') 

## Understanding the results

The result of the fit is saved in ```.hdf5``` file that can be read using any h5 reader (eg. ```h5py``` in python). However, FortesFit has an in-built class to read this file and parse the results for you. FortesFit also provides a variety of plotting functions to visualise and interepret the results of the fits. We will go through all these in this section. 

In [ ]:
# get the plotting library and see the fit result class
from fortesfit import FortesFit_Plots
FortesFit_Plots.FortesFitResult?

# Explanation of some arguments:
# The argument "BurnIn" refers to how many initial chains to ignore (as "burn-in") during the loading of results.
# This is only useful for MCMC methods, but should be kept to 0 if using Nested Sampling
# "old" is an argument to make FortesFit compatible with the previous version. This is False by default and must
# be kept so unless you know specifically why it should change

In [ ]:
fit_filename = f'{source_name}.FortesFit_output.multinest.hdf5'
burnin = 0 # !!!! NOTE: This is 10000 by default. We don't want that !!!!
fit_result = FortesFit_Plots.FortesFitResult(fit_filename, BurnIn = burnin)

# we can now print various information about the fit. for example,
print(f'Source name: {fit_result.objectname}')
print(f'Redshift: {fit_result.redshift}')
print(f'List of free parameters: {fit_result.fit_parameter_names}')
print(f'Best-fit (median) parameter values (including fixed parameters): {fit_result.bestfit_parameters}')
print(f'All parameter chains: {fit_result.chains}')

# Question: What other attributes and methods does FortesFitResult object have?
# Hint: Write fit_result. and Press TAB to see all available options below

While this is handy for advanced analysis, FortesFit already has plotting functions for most common analysis/diagnosis. These plotting functions do not require you to manually load results like we just did. Let us look at how to use these function. We will first plot the best-fit SEDs to see how they compare with the actual data.

In [ ]:
FortesFit_Plots.PlotModelSEDs?
# What are the arguments to this function?

In [ ]:
FortesFit_Plots.PlotModelSEDs(fit_filename, BurnIn = 0) # !!!! DO NOT forget to include BurnIn=0 !!!!

**What am I seeing here?**

The filled black points in the figure are all the observed fluxes. The open red points are the best fit model photometry that is interpolated between the parameter grid points on which the original model is defined. Note that some of these open points might be completely hidden behind the solid points, but each observed flux has an associated model flux even if it is not clearly visible in the figure. The solid lines represent the template SED closest to the original parameter grid on which the models were defined. Specifically, solid black line is the total model, and coloured lines are individual SED models. While these lines serve as a guide to the eye, the true assessment of the quality of fit should be done by comparing filled points and open points. Finally, the shaded region shows the scatter around best-fit template by sampling 100 points from the joint posterior distribution.

We can now take a look at a more quantitative measure of the fit by visualising all the free parameters in a corner plot.

In [ ]:
FortesFit_Plots.SummaryFigures?

In [ ]:
FortesFit_Plots.SummaryFigures(fit_filename, BurnIn = 0) # !!!! DO NOT forget to include BurnIn=0 !!!!

**What am I seeing here?**

The first figure is the progression of the sampling for each parameter. This is useful to understand whether each parameter chain has burnt in or not. We will ignore this plot since it is not very relevant for Nested Sampling.

The second figure is a corner plot. This is a type of plot that shows the posterior distribution of each parameter (along the diagonal) and the correlations between different pairs of parameters.

It is also important to compare the posterior distribution with the prior distribution to quantify the information gained over the prior knowledge. Generally, if the prior and posterior distributions look similar then the parameter is said to be unconstrained by the fit. We can plot prior and posterior distributions for each model in FortesFit.

In [ ]:
FortesFit_Plots.PlotPosteriors?
# Note:
# This function plots posterior and prior distribution seperately for each model and prompts user 
# on whether to go to the next model

In [ ]:
FortesFit_Plots.PlotPosteriors(fit_filename, BurnIn = 0) # !!!! DO NOT forget to include BurnIn=0 !!!!

In [ ]:
!conda env export | grep -v "^prefix: " > environment.yml

**What am I seeing here?**

The shaded gray histogram is the posterior distribution obtained from the fit, while the solid black line is the prior distribution supplied by the user. The description of the model being plotted is given at the top of the figure and the name of the parameter is given at the top of each subplot.

Questions: Which parameters show significant deviation from the priors? Do these values make sense? Would you says these parameters are constrained?

# End of Part 3 !

This is the end of Part 3 of this tutorial. This should equip you with the working knowledge of FortesFit. At the end of this you should be able to (i) Register your own filters, (ii) Register models, (iii) Perform SED fits, and (iv) Interpret the results from the fit.

Writing your model registration script can be tricky in the beginning, so please feel free to write to [write to me](mailto:d.h.liya2@ncl.ac.uk) if you need help writing a script for your own model!

All the best, and happy fitting :)

----

# Advanced usage

This is yet incomplete section of the tutorial. But my aim is to include instructions for the advanced and/or experimental features of FortesFit. Some of these features include parallelisation of various parts to deal with bigger models and large number of sources, simulating observed SEDs from a given model, fitting multiple SEDs simultaneously, and speeding up the fitting using machine learning.

I have made varying levels of progress on each of these goals but none of them are available in the public version of FortesFit. So if you are interested in trying out any of these features or want to collaborate in developing/testing/fiding uses of these or other features then please get in touch!

For now, this section has some additional ```readin()``` scripts (that we have written!) for commonly used models. I would really appreciate if you cited/acknowledged me when you make use of these in your research!

## Stellar population model (Bruzual and Charlot, 2003)

This is a stellar population synthesis framework from [Bruzual & Charlot, 2003](https://ui.adsabs.harvard.edu/abs/2003MNRAS.344.1000B/abstract) which uses observed as well as synthetic stellar spectra. In order to generate the provided templates, we adopted a Chabrier Initial Mass Function (IMF); solar metallicity; and exponentially-decaying star-formation history. 

The model SEDs were generated for 10 different ages (5 Myr, 25 Myr, 100 Myr, 290 Myr, 640 Myr, 900 Myr, 1.4 Gyr, 2.5 Gyr, 5 Gyr, 11 Gyr) and 12 different values of SFR decay timescale (10 Myr, 50 Myr, 100 Myr, 200 Myr, 500 Myr, 750 Myr, 1 Gyr, 2 Gyr, 5 Gyr, 7 Gyr, 10 Gyr, 15 Gyr). These model SEDs were generated for wavelength range 91Å; to 160$\mu m$. 

We then apply the Calzetti attenuation law using [extinction python package](https://extinction.readthedocs.io/en/latest/) with $E(B-V)$ between 0-2 to simulate the attenuation of starlight by the dust in the galaxy, assumed to act as a screen. The free shape parameters of the full stellar population model with dust attenuation are, (i) the age of the galaxy, (ii) SFR decay timescale, and (iii) extinction. The scale parameter for this model is log of stellar mass in the units of solar mass.

In [ ]:
import os
import numpy as np
from fortesfit.FortesFit_Settings import FortesFit_Cosmology as cosmo
import extinction  # Note, this package is not distributed with FortesFit

def readtemplates(templateroot=None):
    """ 
    Read in the templates that will be used by the associated readin routine
        
    The returned variable can have any user-defined form that is understood by the 
    readin and supplements routines below.

    The model template files have the following parameters:
    ages = [5.012E+06, 2.512E+07, 1.015E+08, 2.861E+08, 6.405E+08, 9.048E+08, 1.434E+09, 2.500E+09, 
    5.000E+09, 1.100E+10] Yr
    taus = [0.01, 0.05, 0.1, 0.2, 0.5, 0.75, 1.0, 2.0, 5.0, 7.5, 10.0, 15.0] Gyr 
    """

    if templateroot is None:
        # directory that works on my computer
        templateroot = 

    # From the first file, get the first column, which is the wavelength array. Same for all the model spectra.
    # Using taus in pre-defined order to avoid the problem of sorting them in the end.
    compatibility_taus = [0.01, 0.05, 0.1, 0.2, 0.5, 0.75, 1.0, 2.0, 5.0, 7.5, 10.0, 15.0]
    specfiles = [f'{templateroot}ascii_spectra/bc03_tau_{current_tau}_spec' for current_tau in compatibility_taus]
    
    # Get the ages of the oldest stars from the header of the first spectra file. Will be the same for other files too.
    with open(specfiles[0]) as f:
        for i in range(5):
            readstring = f.readline()

    ages = np.array([float(agestring) for agestring in readstring.split()[3:]]) # Ages in yr

    template_basic = np.loadtxt(specfiles[0]) # in linear Lsun/Angstrom
    wavelength     = template_basic[:,0]/1.0e4 #  base wavelengths are in Angstroms

    mstar_norm = 10.0  # The default stellar mass to normalise to, in log solar masses
    seds = np.zeros((len(wavelength), len(specfiles), len(ages)))   
    
    taus = []
    
    # Also store the SFR as supplied 
    for ifile in range(len(specfiles)):
        template_basic = np.loadtxt(specfiles[ifile]) # in linear Lsun/Angstrom
        tau_prefix = os.path.basename(specfiles[ifile]).split('_')[2]  # The tau value in string form, for file reference
        taus.append(float(tau_prefix))  # Store the tau value as a number

        # Use the BC03 4color output files to get the stellar mass
        infofile = f'{templateroot}derived_quantities/bc03_tau_{tau_prefix}.4color'
        info_basic = np.loadtxt(infofile)
        info_age   = info_basic[:,0]  #  Age of the SSPs is the first column
        info_mstar = info_basic[:,6]  #  Mass in stars, excluding remnants

        for iage, age_val in enumerate(ages):
            mstar = np.interp(np.log10(age_val), info_age, info_mstar)
            scale_factor = 10**(mstar_norm)/mstar
            # Convert SED from Lsun/Angstrom to lambdaLlambda in erg/s units
            seds[:, ifile, iage] = template_basic[:, iage+1]*3.826e33*template_basic[:, 0]*scale_factor
        
    parameters = {'tau_Gyr': np.array(taus), 'age_Gyr': ages/1e9, 'logStellarMass': mstar_norm} 

    templates = {'wavelength':wavelength, 'seds':seds, 'parameters':parameters}
    
    return templates

def readin(parameters, redshift, templates=None):
    """ 
    Given a specific parameter set and a redshift, get the BC03 tau model spectrum with Calzetti law extinction.

    The parameters are :
            logStellarMass: The total mass in stars in log solar masses
            age_Gyr: the age of the oldest stars in log Gyr
            tau_Gyr: the exponential decay timescale of the stellar population in log Gyr
            E_BV_czt: B-V colour excess
            
    Returns the sed as a dictionary with wavelength in microns and flux in erg/s/cm^2/micron
    """
    
    # Templates are needed to process the readin for this class of model. Catch cases where the template is not supplied.
    if templates == None:
        raise ValueError('Templates must be supplied')
    # These templates are already normalised to 10^10 solar masses and in nuLnu units.

    wave_basic = templates['wavelength'] # rest wavelength in microns (see readtemplates)
    wave = wave_basic*(1.0+redshift)  # Observed wavelength 

    # identify the index corresponding to the age parameter
    tau_index = np.argmin(np.abs(templates['parameters']['tau_Gyr'] - parameters['tau_Gyr']))
    age_index = np.argmin(np.abs(templates['parameters']['age_Gyr'] - parameters['age_Gyr']))

    # get the correct template
    template_orig = templates['seds'][:, tau_index, age_index]

    # it doesn't matter is we apply this scaling before or after extinction. I will do it after
    scale_factor = 10**(parameters['logStellarMass'] - templates['parameters']['logStellarMass']) # Stellar mass offset from 10 log erg/s

    # Apply dust extinction to the stellar light
    # r_v = 4.05 +- 0.8 (Calzetti+00, ApJ 533, 682)
    r_v = 4.05
    a_v = r_v*parameters['E_BV_czt']
    attenuation = extinction.calzetti00(wave_basic*1e4, a_v = a_v, r_v = r_v) # extinction takes wavelength in Angstroms
    template_attenuated = extinction.apply(attenuation, template_orig)

    lfactor = 4.0*np.pi*(cosmo.luminosity_distance(redshift).value*3.0856e24)**2.0 # area of spehere in cm^2
    observedflux = (template_attenuated*scale_factor/lfactor)/wave

    sed = {'observed_wavelength': wave, 'observed_flux': observedflux}
    
    return sed

This readin script and the associated SED templates are distributed with this tutorial. Please change the line ```templateroot = ''``` to point to the full path of where you store the templates after extraction on your computer.

The recommended registration line for this model is as follows,

In [ ]:
# filterids = 
bc03modelid = FortesFit_ModelManagement.register_model('bc03_readin', {'logStellarMass': 10.0, 
        'E_BV_czt': np.arange(0, 2, 0.07), 
        'tau_Gyr': np.array([0.01, 0.05, 0.1, 0.2, 0.5, 0.75, 1, 2, 5, 7, 10, 15]), 
        'age_Gyr': np.array([5.012e-03, 2.512e-02, 1.015e-01, 2.861e-01, 6.405e-01, 9.048e-01, 1.434, 2.5, 5, 11])},
        'logStellarMass', description='BC03 Tau (chabrier, solartau, calzetti), 12/06/2023, David+Devang', 
        filterids=filterids)